# NASA Common Metadata Repository

[NASA's Common Metadata Repository (CMR)]{https://www.earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr} is a high-performance, high-quality, continuously evolving metadata system that catalogs all data and service metadata records for NASA's Earth Observing System Data and Information System (EOSDIS) system and will be the authoritative management system for all EOSDIS metadata. These metadata records are registered, modified, discovered, and accessed through programmatic interfaces leveraging standard protocols and APIs.

You can search the CMR system online here: https://cmr.earthdata.nasa.gov/

In [ ]:
import rsgislib.dataaccess.nasa_cmr
import os
import datetime
import pprint

## Get information on a Product

You can query the CMR for metadata on a product using the function below. You can look up the available products and their names here:
https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-standard-products


In [ ]:
# GEDI Canopy Product
prod_name="GEDI02_B"

In [ ]:
info_dict = rsgislib.dataaccess.nasa_cmr.get_prods_info(prod_short_name=prod_name)


In [ ]:
pprint.pprint(info_dict)

## Find Most Recent Version

You can find the most recent version of the product you are interested in using the function below:

In [ ]:
max_version = rsgislib.dataaccess.nasa_cmr.get_max_prod_version(prod_short_name=prod_name)
max_version

## Search for Data

Use the following function to search for the data you are interested in. You can specify the boundary box but also limit the search by date

In [ ]:
bbox = [39.1, 39.5, -8, -7.6]
prod_data_granules = rsgislib.dataaccess.nasa_cmr.find_granules(prod_short_name=prod_name, 
                                                                version=max_version, 
                                                                only_dnwld = True, 
                                                                bbox = bbox, 
                                                                pt = None, 
                                                                start_date = datetime.datetime(year=2020, month=4, day=1), 
                                                                end_date = datetime.datetime(year=2020, month=4, day=30), 
                                                                cloud_min = 0, 
                                                                cloud_max = None, 
                                                                sort_date = True, 
                                                                sort_desc = True, 
                                                                page_size = 100, 
                                                                page_num = 1, 
                                                                other_params = None)


In [ ]:
# Print out the number of data granules found
len(prod_data_granules)

### Print a single product to check the data type to download. 

Look up the file that you will want to download (e.g., ATL08_20200517210904_07990708_006_01.h5) and note the data type (e.g., application/x-hdf5) as you'll need it later.

In [ ]:
pprint.pprint(prod_data_granules[0])

## How much data is to be downloaded?

The following function sums the file sizes of the identified data to download:

In [ ]:
total_file_size = rsgislib.dataaccess.nasa_cmr.get_total_file_size(granule_lst=prod_data_granules)
total_file_size

## Get the Download URLs

The next step is to build a [pysondb](https://github.com/pysonDB/pysonDB) database of the data file to be downloaded

In [ ]:
# The file name of the pysondb JSON database
db_file_name = f"{prod_name}_scns_db.json"

In [ ]:
urls_not_found = rsgislib.dataaccess.nasa_cmr.create_cmr_dwnld_db(
    db_json = db_file_name, 
    granule_lst=prod_data_granules, 
    dwnld_file_mime_type = "application/x-hdfeos"
)

In [ ]:
# Check whether there were data for which the URL could not be found.
len(urls_not_found)

In [ ]:
urls_not_found

## Download the Data

### Create Username and Password File

You need to be careful with your username and password and therefore you should not write them into the notebook. RSGISlib provide a tool/functions for doing a basic encoding of the username and password so they are not stored as free text (Note. be careful as the simple encoding is not secure). 

To create the encoded file, you can use the command line tool `rsgisuserpassfile.py` as shown below:

`rsgisuserpassfile.py userinfo.txt`

Once you have created the `userinfo.txt` file, you can read it into your notebook/script using the `get_username_password` function shown below:

### Create Output Directory

In [ ]:
out_dwnld_dir = "GEDI02_B_data"
if not os.path.exists(out_dwnld_dir):
    os.mkdir(out_dwnld_dir)

### Run Download

In [ ]:
rsgislib.dataaccess.nasa_cmr.download_granules_use_dwnld_db(
    db_json = db_file_name, 
    out_path = out_dwnld_dir, 
    user_pass_file = "userinfo.txt", 
    use_wget = False
)